In [5]:

# filename1= 'Triage-Counterfactual_CoT_gpt-4o_json0_99_20241125_173640.csv'
# filename2 = 'Triage-Counterfactual_CoT_gpt-4o_json100_1000_20241124_173026.csv'
# filename3 = 'Triage-Counterfactual_CoT_gpt-4o_json1000_3000_20241124_211048.csv'
# filepaths = [
#     '../results/Triage-Counterfactual/' + filename1,
#     '../results/Triage-Counterfactual/' + filename2,
#     '../results/Triage-Counterfactual/' + filename3
# ]

# # Read and combine all dataframes
# dfs = [pd.read_csv(filepath) for filepath in filepaths]
# dfs[2] = dfs[2].loc[1:len(dfs[2])-1]
# combined_df = pd.concat(dfs, ignore_index=True)

# # Save the combined dataframe to a new file if needed
# output_filepath = '../results/Triage-Counterfactual/Triage-Counterfactual_CoT_gpt-4o_json0_3000.csv'
# combined_df.to_csv(output_filepath, index=False)

### Create Embeddings for KATE Few Shot Prediction

In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import numpy as np
from tqdm import tqdm

/opt/homebrew/anaconda3/envs/llm-medical-bias/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
data = pd.read_csv('./data/mimic-iv-private/triage_stratified_training.csv')

In [3]:
data['chiefcomplaint'] = data['chiefcomplaint'].astype(str)

# Load the pre-trained SentenceTransformer model for clinical text
# This model is fine-tuned for biomedical and clinical text embeddings
model_name = 'pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb'
model = SentenceTransformer(model_name)

# from transformers import AutoTokenizer, AutoModel
# tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
# model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")


In [31]:
# List of vital signs columns
vital_signs_cols = ['temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'pain']

# Ensure vital signs are numeric and handle missing values
data[vital_signs_cols] = data[vital_signs_cols].apply(pd.to_numeric, errors='coerce')

# Normalize vital signs using Min-Max scaling
scaler = MinMaxScaler()
vital_signs_normalized = scaler.fit_transform(data[vital_signs_cols])

In [33]:
vital_signs_normalized.shape

(386776, 7)

In [34]:
print(f"NaN in embeddings: {np.isnan(vital_signs_normalized).any()}")


NaN in embeddings: True


In [30]:
len(data['chiefcomplaint'].unique())

56073

In [40]:
# Process chief complaints in batches of 10,000
batch_size = 100
symptom_embeddings = []

print("Computing symptom embeddings in batches...")
for i in tqdm(range(0, 100, batch_size)):
    batch = data['chiefcomplaint'].iloc[i:i + batch_size].tolist()
    print(batch)
    embeddings = query(batch)
    symptom_embeddings.extend(embeddings)

    # Save intermediate embeddings
    np.save(f'./data/mimic-iv-private/symptom_embeddings_batch_{i//batch_size}.npy', np.array(embeddings))

# Convert symptom embeddings to a numpy array
symptom_embeddings = np.array(symptom_embeddings)

# Perform PCA on symptom embeddings to reduce dimensionality to 10
print("Performing PCA on symptom embeddings...")
pca = PCA(n_components=10)
symptom_embeddings_reduced = pca.fit_transform(symptom_embeddings)

# Save the reduced embeddings in batches of 10,000
for i in range(0, len(symptom_embeddings_reduced), batch_size):
    batch_reduced = symptom_embeddings_reduced[i:i + batch_size]
    np.save(f'./data/mimic-iv-private/symptom_embeddings_reduced_batch_{i//batch_size}.npy', batch_reduced)

# Concatenate reduced symptom embeddings and vital signs to create comprehensive embeddings
print("Creating comprehensive embeddings...")
comprehensive_embeddings = np.hstack((symptom_embeddings_reduced, vital_signs_normalized))

# Save comprehensive embeddings in batches of 10,000
for i in range(0, len(comprehensive_embeddings), batch_size):
    batch_comprehensive = comprehensive_embeddings[i:i + batch_size]
    np.save(f'./data/mimic-iv-private/comprehensive_embeddings_batch_{i//batch_size}.npy', batch_comprehensive)

Computing symptom embeddings in batches...


  0%|          | 0/1 [00:00<?, ?it/s]

['R Ankle pain', 'Headache', 'Diarrhea, Presyncope', 'RLQ abdominal pain', 'SOB', 'Agitation, SI', 'Headache', 'Back pain', 'Buttock pain, L Leg pain', 'R Hand injury', 'KIDNEY STONES', 'Fever, Cough', 'N/V, RENAL FAILURE', 'Abd pain', 'Laceration, Altered mental status', 'Chest pain, Dyspnea', 'Abd pain, Abnormal ultrasound', 'SYNCOPE', 'n/v/d', 'Chest pain, Dyspnea', 'Abd pain, s/p colonoscopy', 'MEDICAL DEVICE PROBLEM', 'Chest pain', 'Epigastric pain', 'SI', 'BACK PAIN', 'HTN', 'RLQ ABDOMINAL PAIN', 'Buttock pain, Pelvic pain', 'MS CHANGES', 'Body pain', 'RT ARM PAIN/SWELLING', 'Altered mental status', 'Back pain', 'Dyspnea', 'L Shoulder pain', 'Lip swelling', 'Wound eval, Transfer', 'SHORTNESS OF BREATH', 'S/P MVC', 'ABNL LABS/THROMBOCYTOPENIA', 'R Hand injury, Syncope', 'Chest pain, Cough', 'Dyspnea, Leg swelling', 'R Wrist injury, s/p Fall', 'L Leg numbness, s/p Fall', 'LEFT KNEE PAIN', 'Abd pain, Nausea', 'Rectal pain, Abscess', 'ETOH', 'Lower back pain', 'LEFT SHOULDER PAIN', '

100%|██████████| 1/1 [00:26<00:00, 26.98s/it]


Performing PCA on symptom embeddings...
Creating comprehensive embeddings...


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 100 and the array at index 1 has size 386776

In [64]:
vital_signs_normalized.shape
symptom_embeddings_reduced.shape

(100, 10)

In [55]:
embeddings = np.load('./data/mimic-iv-private/symptom_embeddings_batch_0.npy', allow_pickle=True)
similarities = model.similarity(embeddings, embeddings)


In [60]:
# Iterate through the first 100 rows
num_samples = 100
for i in range(num_samples):
    # Get the similarity scores for the current sample
    similarity_row = similarities[i]

    # Exclude self-similarity by setting its score to a very low value
    similarity_row[i] = -np.inf

    # Find the index of the most similar sample
    closest_index = np.argmax(similarity_row)

    # Retrieve and print the symptoms
    symptom = data['chiefcomplaint'].loc[i]
    closest_symptom = data['chiefcomplaint'].loc[closest_index.item()]

    print(f"Sample {i}: {symptom}")
    print(f"Most similar symptom: {closest_symptom}")
    print()

Sample 0: R Ankle pain
Most similar symptom: R Flank pain

Sample 1: Headache
Most similar symptom: Headache

Sample 2: Diarrhea, Presyncope
Most similar symptom: Diarrhea, Weakness

Sample 3: RLQ abdominal pain
Most similar symptom: RLQ ABDOMINAL PAIN

Sample 4: SOB
Most similar symptom: SI

Sample 5: Agitation, SI
Most similar symptom: SI

Sample 6: Headache
Most similar symptom: Headache

Sample 7: Back pain
Most similar symptom: BACK PAIN

Sample 8: Buttock pain, L Leg pain
Most similar symptom: Buttock pain, Pelvic pain

Sample 9: R Hand injury
Most similar symptom: R Hand injury, Syncope

Sample 10: KIDNEY STONES
Most similar symptom: Dysuria

Sample 11: Fever, Cough
Most similar symptom: Dyspnea, Fever

Sample 12: N/V, RENAL FAILURE
Most similar symptom: N/V

Sample 13: Abd pain
Most similar symptom: ABD PAIN

Sample 14: Laceration, Altered mental status
Most similar symptom: Altered mental status

Sample 15: Chest pain, Dyspnea
Most similar symptom: Chest pain, Dyspnea

Sample 

In [ ]:

# Save the final combined embeddings to CSV
print("Saving final comprehensive embeddings to CSV...")
comprehensive_embeddings_df = pd.DataFrame(comprehensive_embeddings)
comprehensive_embeddings_df.to_csv('./data/mimic-iv-private/comprehensive_embeddings.csv', index=False)

print("Process completed successfully!")

## Compute Embeddings

In [70]:
symptom_embeddings = model.encode(
    data['chiefcomplaint'][0],
    batch_size=1,
    convert_to_numpy=True
)

In [21]:
# Compute embeddings for 'chiefcomplaint'
print("Computing symptom embeddings...")
symptom_embeddings = model.encode(
    data['chiefcomplaint'].tolist(),
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True
)


Computing symptom embeddings...


Batches: 100%|██████████| 6044/6044 [06:17<00:00, 16.00it/s]


In [66]:
symptom_embeddings

array([[ 6.41967118e-01, -3.26322138e-01,  3.70455176e-01, ...,
        -3.82121503e-01,  4.87686008e-01,  5.56469858e-02],
       [-2.16401219e-01,  4.29060668e-01,  1.29151508e-01, ...,
         4.02796656e-01,  1.16578601e-01,  3.77998918e-01],
       [-8.46060067e-02,  4.77421254e-01,  1.36317328e-01, ...,
         1.68062791e-01,  1.85714006e-01,  4.71749753e-01],
       ...,
       [ 2.08412111e-01,  1.22647863e-02,  3.20407689e-01, ...,
        -2.86299825e-01,  1.05803594e-01, -4.17001396e-01],
       [ 5.94735138e-05,  1.06321789e-01,  1.06389306e-01, ...,
         2.07356755e-02,  8.84340584e-01, -2.74949253e-01],
       [ 5.79318523e-01, -5.16417861e-01, -1.82688478e-02, ...,
        -4.23944563e-01, -2.11833194e-01,  6.18369460e-01]], dtype=float32)

In [67]:
np.save('./data/mimic-iv-private/symptom_embeddings.npy', symptom_embeddings)


In [26]:
# Perform PCA on symptom embeddings to reduce dimensionality to 30
print("Performing PCA on symptom embeddings...")
pca = PCA(n_components=14)
symptom_embeddings_reduced = pca.fit_transform(symptom_embeddings)

Performing PCA on symptom embeddings...


In [ ]:
# embeddings = np.load('./data/mimic-iv-private/comprehensive_embeddings.npy',allow_pickle=True)

In [63]:
import numpy as np
import pandas as pd
from collections import Counter


def compute_cosine_similarity(embedding, embeddings, epsilon=1e-8):
    """
    Compute cosine similarity between a single embedding and a set of embeddings.
    Avoids division by zero by adding epsilon to the norm.
    """
    # Compute cosine similarities
    similarities = np.dot(embeddings, embedding)
    return similarities

def get_top_k_similar(embedding, embeddings, k=5):
    """
    Find the top-k most similar samples to a given embedding.
    """
    similarities = compute_cosine_similarity(embedding, embeddings)
    top_k_indices = np.argsort(similarities)[-k:][::-1]
    return top_k_indices, similarities[top_k_indices], similarities

patient = 8
# Get top 5 similar embeddings
top_k_indices, top_k_similarities, similarities = get_top_k_similar(symptom_embeddings[patient], symptom_embeddings, k=25)

# Retrieve and print the corresponding symptoms
print(f"Querying {data['chiefcomplaint'].iloc[patient]} most similar samples:")
for i, index in enumerate(top_k_indices):
    symptom = data['chiefcomplaint'].iloc[index]
    acuity = data['acuity'].iloc[index]
    similarity = top_k_similarities[i]
    print(f"Rank {i+1}: {symptom} | Index: {index} | Similarity: {similarity:.4f} | Acuity: {acuity}")


Querying Buttock pain, L Leg pain most similar samples:
Rank 1: Buttock pain, L Leg pain | Index: 8 | Similarity: 182.3298 | Acuity: 3.0
Rank 2: Buttock pain, L Leg pain | Index: 123029 | Similarity: 182.3297 | Acuity: 3.0
Rank 3: BUTTOCK, LEG PAIN | Index: 235208 | Similarity: 175.0675 | Acuity: 3.0
Rank 4: R BUTTOCK/LEG PAIN | Index: 246717 | Similarity: 172.5656 | Acuity: 3.0
Rank 5: Buttock pain, Leg pain | Index: 302301 | Similarity: 172.5599 | Acuity: 4.0
Rank 6: Leg pain, L Leg pain | Index: 364963 | Similarity: 170.6013 | Acuity: 3.0
Rank 7: Body pain, L Leg pain | Index: 114039 | Similarity: 169.9877 | Acuity: 3.0
Rank 8: Buttock pain, R Leg pain | Index: 282507 | Similarity: 169.9574 | Acuity: 3.0
Rank 9: Buttock pain, R Leg pain | Index: 226989 | Similarity: 169.9574 | Acuity: 3.0
Rank 10: L BUTTOCK PAIN | Index: 345878 | Similarity: 168.6251 | Acuity: 1.0
Rank 11: L BUTTOCK PAIN | Index: 100046 | Similarity: 168.6251 | Acuity: 4.0
Rank 12: L BUTTOCK PAIN | Index: 331936 | S

In [64]:
top_k_indices

array([     8, 123029, 235208, 246717, 302301, 364963, 114039, 282507,
       226989, 345878, 100046, 331936,  46441, 189041, 273005, 171619,
        29373, 163178, 248571, 111618, 166571, 322698, 311573, 165287,
       230436])

Now, let's add the vitals and repeat the analysis -- let's first try cosine

In [65]:

# Example vital sign columns in your dataframe
vital_sign_columns = ['temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'pain']

# Extract the query sample's vital signs
query_vital_signs = data.loc[top_k_indices[0], vital_sign_columns].values.astype(float)

# Extract the vital signs for the top 25 samples
top_25_vital_signs = data.loc[top_k_indices, vital_sign_columns].values.astype(float)

# Normalize the query and top 25 vital signs
query_vital_signs_normalized = query_vital_signs / np.linalg.norm(query_vital_signs)
top_25_vital_signs_normalized = top_25_vital_signs / np.linalg.norm(top_25_vital_signs, axis=1, keepdims=True)

# Compute cosine similarity
vital_signs_similarity = np.dot(top_25_vital_signs_normalized, query_vital_signs_normalized)

# Get the index of the most similar sample based on vital signs
most_similar_index = np.argmax(vital_signs_similarity)
most_similar_sample = data.loc[top_k_indices[most_similar_index]]

# Retrieve the top-k similar samples based on vital signs
top_k_vital_indices = np.argsort(vital_signs_similarity)[-5:][::-1]
top_k_vital_samples = data.loc[top_k_indices[top_k_vital_indices]]

# Output results
print(f"Most similar sample (based on vital signs):")
print(most_similar_sample)

print("\nTop 5 most similar samples (based on vital signs):")
print(top_k_vital_samples)

Most similar sample (based on vital signs):
subject_id                        16052047
stay_id                           36556660
temperature                           98.5
heartrate                             58.0
resprate                              14.0
o2sat                                100.0
sbp                                  151.0
dbp                                   70.0
pain                                   6.0
acuity                                 3.0
chiefcomplaint    Buttock pain, L Leg pain
Name: 8, dtype: object

Top 5 most similar samples (based on vital signs):
        subject_id   stay_id  temperature  heartrate  resprate  o2sat    sbp  \
8         16052047  36556660         98.5       58.0      14.0  100.0  151.0   
246717    17010538  31731207         98.4       60.0      18.0  100.0  164.0   
163178    16531643  36827756         98.4       63.0      19.0  100.0  142.0   
235208    17113027  31142049         98.3       71.0      16.0  100.0  138.0   
165287  

In [55]:
# Example vital sign columns in your dataframe
vital_sign_columns = ['temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'pain']

# Extract the query sample's vital signs
query_vital_signs = data.loc[top_k_indices[0], vital_sign_columns].values.astype(float)

# Extract the vital signs for the top 25 samples
top_25_vital_signs = data.loc[top_k_indices, vital_sign_columns].values.astype(float)

# Compute Euclidean distance between the query and the top 25 samples
distances = np.linalg.norm(top_25_vital_signs - query_vital_signs, axis=1)

# Get the index of the most similar sample based on Euclidean distance
most_similar_index = np.argmin(distances)
most_similar_sample = data.loc[top_k_indices[most_similar_index]]

# Retrieve the top-k similar samples based on Euclidean distance
top_k_vital_indices = np.argsort(distances)[:5]
top_k_vital_samples = data.loc[top_k_indices[top_k_vital_indices]]

# Output results
print(f"Most similar sample (based on Euclidean distance):")
print(most_similar_sample)

print("\nTop 5 most similar samples (based on Euclidean distance):")
print(top_k_vital_samples)

Most similar sample (based on Euclidean distance):
subject_id            15289989
stay_id               36797979
temperature               97.9
heartrate                 92.0
resprate                  20.0
o2sat                    100.0
sbp                      143.0
dbp                       82.0
pain                      10.0
acuity                     3.0
chiefcomplaint    R Ankle pain
Name: 0, dtype: object

Top 5 most similar samples (based on Euclidean distance):
        subject_id   stay_id  temperature  heartrate  resprate  o2sat    sbp  \
0         15289989  36797979         97.9       92.0      20.0  100.0  143.0   
62199     12618057  37799336         98.6       91.0      16.0  100.0  148.0   
224967    13524705  32250368         97.8       85.0      18.0   98.0  150.0   
222528    12672071  31541890         96.6      103.0      18.0   99.0  146.0   
222646    19456368  35401303         98.4       80.0      20.0  100.0  146.0   

         dbp  pain  acuity chiefcomplaint  
0

## Run more scripts

In [ ]:
!python llm-predict-triage.py --dataset Triage-Private-Stratified --start 0 --end 0 --model openai-gpt-4o-chat --strategy KATE --json --debug --k_shots 10

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading Dataset...
Potentially loading existing predictions...
Making 1 new predictions...
0it [00:00, ?it/s]Estimate the patient's acuity from 1-5 based on the following guidelines: Acuity is assessed using the Emergency Severity Index (ESI) Five Level triage system. This priority is assigned by a registered nurse. Level 1 is the highest priority, while level 5 is the lowest priority. Answer in valid JSON format, providing acuity as a single numeric value in the key 'acuity'.

Input: temperature   heartrate   resprate   o2sat   sbp   dbp   pain   chiefcomplaint
99.3   92.0   24.0   96.0   134.0   57.0   0   Dyspnea on exertion
Output: 2.0

Input: temperature   heartrate   resprate   o2sat   sbp   dbp   pain   chiefcomplaint
98.3   111.0   16.0   100.0   125.0   60.0   0   Dyspnea on exertion
Output: 2.0

Input: temperature   heartrate   resprate   o2sat   sbp   dbp   pain   chiefcomplaint
98.2   72.0   18.0   100.0   134.0   72.0   0   Dyspnea on exertion
Output: 2.0

Input: temperatu

In [84]:
!python llm-evaluate-triage.py --dataset Triage-Private-Stratified --start 0 --end 2499 --ordinal --parameters Triage-Private-Stratified_FewShot_openai-gpt-4o-chat_json0_2500_20241127_230336 --by_class

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading Dataset...
2500
Loading Predictions...
Overall Metrics: {'overall': {'accuracy': 0.566, 'precision': 0.6377718494209463, 'recall': 0.566, 'f1_score': 0.5786089315616268, 'adjusted_accuracy': 0.9564, 'adjusted_precision': 0.9679252936610608, 'adjusted_recall': 0.9564, 'adjusted_f1': 0.9593395733899968, 'mae': 0.4796, 'mse': 0.5748, 'quadratic_kappa': 0.4725431743783277}, 'by_class': {'1.0': {'precision': 0.6666666666666666, 'recall': 0.046511627906976744, 'f1-score': 0.08695652173913043, 'support': 86.0}, '2.0': {'precision': 0.5937149270482603, 'recall': 0.6320191158900836, 'f1-score': 0.6122685185185185, 'support': 837.0}, '3.0': {'precision': 0.7173708920187793, 'recall': 0.548456568557071, 'f1-score': 0.6216436126932465, 'support': 1393.0}, '4.0': {'precision': 0.2261904761904762, 'recall': 0.6440677966101694, 'f1-score': 0.33480176211453744, 'support': 177.0}, '5.0': {'precision': 0.11764705882352941, 'recall': 0.5714285714285714, 'f1-score': 0.1951219512195122, 'support': 

In [85]:
df = pd.read_csv('./results/Triage-Private-Stratified/Triage-Private-Stratified_ZeroShot_openai-gpt-4o-chat_json_detailed0_1000_20241125_170724.csv')

In [93]:
df

,prompt,Estimated_Acuity,Reasoning,subject_id,stay_id,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,chiefcomplaint
0,Here is the profile of a patient:\n\ntemperatu...,3,NaN,18474069,30615360,98.2,71.0,18.0,94.0,92.0,36.0,0,3.0,Dyspnea on exertion
1,NaN,4,NaN,10482402,30835613,97.3,68.0,18.0,100.0,131.0,74.0,5,3.0,S/P FALL
2,NaN,2,NaN,11668089,30163418,97.6,105.0,22.0,100.0,147.0,76.0,3,2.0,"Chest pain, Dyspnea"
3,NaN,2,NaN,17170624,35921297,97.6,110.0,16.0,98.0,99.0,65.0,10,3.0,Abnormal CT
4,NaN,5,NaN,17532289,37034357,97.7,85.0,20.0,100.0,134.0,76.0,0,4.0,Rash
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,NaN,2,NaN,12003826,33426441,98.8,98.0,18.0,98.0,159.0,91.0,10,3.0,Back pain
696,NaN,2,NaN,17125290,33272361,97.9,100.0,18.0,99.0,105.0,83.0,8,2.0,Inguinal pain
697,NaN,3,NaN,15964158,32546263,98.6,75.0,12.0,98.0,127.0,86.0,unable,3.0,"ETOH, s/p Fall"
698,NaN,2,NaN,13528930,37977947,98.0,120.0,16.0,100.0,156.0,66.0,0,3.0,"WEAKNESS, FATIGUE"


In [88]:
filename ='Triage-Private-Stratified_ZeroShot_openai-gpt-4o-chat_json_detailed600_1000_20241126_024228.csv'
df2 = pd.read_csv(f'./results/Triage-Private-Stratified/{filename}')
df2

,Estimated_Acuity,Reasoning,subject_id,stay_id,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,chiefcomplaint
0,2,NaN,18159815,31608693,99.7,90.0,18.0,98.0,118.0,77.0,0,2.0,"Allergic reaction, Dyspnea"
1,2,NaN,14421355,32098882,99.4,132.0,16.0,100.0,120.0,72.0,7,1.0,"Headache, Sore throat"
2,4,NaN,18505007,36748085,98.6,96.0,18.0,100.0,137.0,90.0,0,4.0,"Dysuria, Urinary frequency"
3,3,NaN,17113137,38284564,98.9,102.0,22.0,96.0,125.0,75.0,0,2.0,___
4,3,NaN,15675253,30288651,98.1,72.0,16.0,100.0,127.0,56.0,5,3.0,"Back pain, MVC"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,2,NaN,17089649,38944330,98.8,114.0,15.0,100.0,148.0,102.0,10,4.0,L Hip pain
397,3,NaN,19343822,39796457,97.2,82.0,16.0,97.0,129.0,57.0,4,3.0,"Abd pain, Chest pain, N/V"
398,3,NaN,12352918,33269944,98.7,106.0,16.0,100.0,136.0,76.0,5,3.0,"MVC, Neck pain"
399,4,NaN,13673417,35894831,99.5,73.0,16.0,97.0,124.0,53.0,3,3.0,LEFT ANKLE PAIN


In [90]:
filename ='Triage-Private-Stratified_ZeroShot_openai-gpt-4o-chat_json_detailed1000_2500_20241127_160229.csv'
df3 = pd.read_csv(f'./results/Triage-Private-Stratified/{filename}')
df3

,Estimated_Acuity,Reasoning,subject_id,stay_id,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,chiefcomplaint
0,3,NaN,19481752,35968442,98.7,114.0,18.0,98.0,130.0,82.0,0,3.0,"Epigastric pain, N/V"
1,5,NaN,17148408,36757338,98.5,58.0,18.0,100.0,104.0,48.0,0,2.0,DIZZINESS
2,2,NaN,18676703,32629111,97.6,103.0,22.0,100.0,237.0,119.0,10,2.0,"N/V, Hyperglycemia"
3,2,NaN,13247319,30478070,98.3,82.0,18.0,99.0,147.0,92.0,0,3.0,CHEST FLUTTERING
4,3,NaN,12782393,30518271,99.2,86.0,17.0,98.0,119.0,70.0,3,3.0,Abd pain
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,4,NaN,12791002,30386504,98.2,63.0,18.0,100.0,149.0,64.0,2,3.0,WOUND EVAL
1496,2,NaN,12938336,34253493,98.2,88.0,18.0,97.0,143.0,63.0,0,2.0,Hemoptysis
1497,2,NaN,13378145,35205258,98.0,114.0,16.0,99.0,131.0,93.0,0,2.0,Seizure
1498,2,NaN,12351481,39111882,100.0,81.0,20.0,92.0,130.0,56.0,0,2.0,Dyspnea


In [94]:
combined_df = pd.concat([df.loc[:599], df2.loc[:399], df3], ignore_index=True)
combined_df.to_csv('Triage-Private-Stratified_ZeroShot_openai-gpt-4o-chat_json_detailed0_2500_20241127.csv'
)

In [95]:
!python llm-evaluate-triage.py --dataset Triage-Private-Stratified --start 0 --end 2499 --ordinal --parameters Triage-Private-Stratified_ZeroShot_openai-gpt-4o-chat_json_detailed0_2500_20241127

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading Dataset...
2500
Loading Predictions...
Overall Metrics: {'overall': {'accuracy': 0.4172, 'precision': 0.5540308291028618, 'recall': 0.4172, 'f1_score': 0.4214614602283862, 'adjusted_accuracy': 0.882, 'adjusted_precision': 0.9471513342295009, 'adjusted_recall': 0.882, 'adjusted_f1': 0.9119957692487527, 'mae': 0.7208, 'mse': 1.04, 'quadratic_kappa': 0.2621665333269161}}
Evaluation complete. Metrics and plots saved.


In [96]:
!python llm-evaluate-triage.py --dataset Triage-Private-Stratified --start 0 --end 2499 --ordinal --parameters Triage-Private-Stratified_ZeroShot_openai-gpt-4o-chat_json0_2500_20241127_084108

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading Dataset...
2500
Loading Predictions...
Overall Metrics: {'overall': {'accuracy': 0.5532, 'precision': 0.6364463639662771, 'recall': 0.5532, 'f1_score': 0.5643732115298273, 'adjusted_accuracy': 0.922, 'adjusted_precision': 0.9633680383607801, 'adjusted_recall': 0.922, 'adjusted_f1': 0.9373247965516198, 'mae': 0.536, 'mse': 0.7368, 'quadratic_kappa': 0.3668244037012104}}
Evaluation complete. Metrics and plots saved.


### Bias Scripts

In [ ]:
!python llm-predict-triage.py --dataset Triage-Counterfactual --start 0 --end 3000 --model gpt-4o-mini --strategy ZeroShot --bias --json --vitals_off

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading Dataset...
Potentially loading existing predictions...
Making 3001 new predictions...
99it [01:08,  1.20it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_gpt-4o-mini_json0_100_20241128_020646.csv
Saved progress after 100 predictions.
199it [02:17,  1.33s/it]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_gpt-4o-mini_json0_200_20241128_020756.csv
Saved progress after 200 predictions.
299it [03:24,  1.18it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_gpt-4o-mini_json0_300_20241128_020903.csv
Saved progress after 300 predictions.
399it [04:42,  1.44it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_gpt-4o-mini_json0_400_20241128_021020.csv
Saved progress after 400 predictions.
499it [05:53,  1.49it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_gpt-4o-mini_json0_500_20241128_021131.csv
Saved progress 

In [ ]:
!python llm-predict-triage.py --dataset Triage-Counterfactual --start 0 --end 3000 --model gpt-3.5-turbo --strategy ZeroShot --bias --json --vitals_off

In [ ]:
!python llm-predict-triage.py --dataset Triage-Counterfactual --start 0 --end 3000 --model gpt-4o --strategy ZeroShot --bias --json --vitals_off

In [ ]:
!python llm-predict-triage.py --dataset Triage-Counterfactual --start 0 --end 3000 --model claude-3-haiku-20240307 --strategy ZeroShot --bias --json --vitals_off

In [ ]:
!python llm-predict-triage.py --dataset Triage-Counterfactual --start 0 --end 3000 --model claude-3-sonnet-20240229 --strategy ZeroShot --bias --json --vitals_off